In [1]:
import numpy as np
import pandas as pd
import pickle
from itertools import chain, count
import re
from pprint import pprint
from multiprocessing import Pool

In [2]:
dat = pd.read_pickle("../data/full_es.pkl")

In [3]:
dat.head()

,headline,keyfacts,content,tags,time
0,Una jueza del Tribunal Supremo suspende parcia...,[ Decidió atender a los grupos conservadores y...,"El martes, antes de presidir la fiesta de fin ...",[],2014-01-02
1,'La revolución cubana sigue sin compromisos co...,[ 'Jamás hemos cedido ni cederemos ante agresi...,El presidente Raúl Castro reveló que se está i...,[],2014-01-02
2,La NSA trabaja en un ordenador cuántico capaz ...,[ La información proviene de los documentos de...,La Agencia de Seguridad Nacional (NSA) trabaja...,[],2014-01-03
3,Último adiós a la ex Miss Venezuela Mónica Spe...,[ Mónica Spear y su marido fueron asesinados e...,Esta semana Venezuela ha recibido una noticia ...,[],2014-01-10
4,Michoacán pone en jaque al Gobierno de Peña Nieto,[ El Gobierno envía más policías y militares y...,La situación en el Estado mexicano de Michoacá...,[],2014-01-14


In [4]:
dat.shape

(37702, 5)

In [5]:
entities = pickle.load(open("../data/es_processed_entities.pkl", "rb"))

In [6]:
len(entities)

37702

In [7]:
dat['entities'] = 0
dat['entities'].astype('object', inplace=True)
for i, ents in entities.items():
    dat['entities'].iloc[int(i)] = ents

/home/mithrillion/anaconda3/envs/py36/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [25]:
def process_story(row):
    story = row['content']
    if len(story) < 50:
        return None, None, None
    facts = row['keyfacts']
    entities = row['entities']
    # get elligible entities
    # check if any entity in each fact
    elligible_facts = []
    flat_mentions = list(chain.from_iterable([mention for mention in entity['mentions']] for entity in entities))
    for fact in facts:
        if np.any([re.search(re.escape(mention), fact) is not None for mention in flat_mentions]):
            elligible_facts.append(fact)
            continue
    # anonymise story and facts
    if len(elligible_facts) > 0:
        for i, entity in enumerate(sorted(entities, key=lambda x: len(x['entity_name']), reverse=True)):
            for mention in sorted(entity['mentions'], key=len, reverse=True):
                story = re.sub(re.escape(mention), '@entity{0}'.format(i), story)
                for j in range(len(elligible_facts)):
                    elligible_facts[j] = re.sub(re.escape(mention), '@entity{0}'.format(i), elligible_facts[j])
        # fix "glued" entities
        story = re.sub(r'(?<=[^\s\"\'])@entity(\d+)', r' @entity\1', story)
        for i in range(len(elligible_facts)):
            elligible_facts[i] = re.sub(r'(?<=[^\s\"\'])@entity(\d+)', r' @entity\1', elligible_facts[i])
        # output
        questions = []
        answers = []
        for fact in elligible_facts:
            for ent in set(re.findall(r'@entity\d+', fact)):
                if re.search(re.escape(ent) + '(?=[ .,?!:\[\];\-"\'])', story):
                    questions.append(re.sub(re.escape(ent), r'@placeholder', fact))
                    answers.append(ent)
        return story, questions, answers
    else:
        return None, None, None

In [26]:
def func_process_story(iterred_row):
    return process_story(iterred_row[-1])

In [27]:
N = 3
print(dat['content'].iloc[N])
print("-" * 20)
print(dat['keyfacts'].iloc[N])
print("-" * 20)
pprint(dat['entities'].iloc[N])

Esta semana Venezuela ha recibido una noticia que ha causado gran consternación en todo el país: el asesinato de la actriz, modelo y ex Miss Venezuela Mónica Spear y de su esposo, el irlandés Thomas Henry Berry . 
El fatídico hecho se produjo en una autopista Venezolana cuando el matrimonio se disponía a regresar a su casa. En el interior del vehículo, se hallaron los cuerpos sin vida de la joven pareja y a la hija de ambos, de 5 años que también resultó herida.
Miles de personas acudieron al velatorio de la pareja. Tanto artistas como familiares y allegados de la actriz y el empresario europeo acudieron desde el mediodía al Cementerio del Este de Caracas para dar el último adiós a la pareja.
Seguidores de la exreina de belleza se aglomeraron en las inmediaciones de la capilla ardiente, en medio de una gran expectación y con presencia de periodistas locales y extranjeros.
La larga fila de personas que se acercaron para dar el pésame a la familia era testigo del paso de decenas de coron

In [28]:
process_story(dat.iloc[N])

('Esta semana @entity12 ha recibido una noticia que ha causado gran consternación en todo el país: el asesinato de la actriz, modelo y ex @entity8 @entity10 y de su esposo, el @entity13 @entity6 . \nEl fatídico hecho se produjo en una autopista Venezolana cuando el matrimonio se disponía a regresar a su casa. En el interior del vehículo, se hallaron los cuerpos sin vida de la joven pareja y a la hija de ambos, de 5 años que también resultó herida.\nMiles de personas acudieron al velatorio de la pareja. Tanto artistas como familiares y allegados de la actriz y el empresario @entity15 acudieron desde el mediodía al @entity0 para dar el último adiós a la pareja.\nSeguidores de la exreina de belleza se aglomeraron en las inmediaciones de la capilla ardiente, en medio de una gran expectación y con presencia de periodistas locales y extranjeros.\nLa larga fila de personas que se acercaron para dar el pésame a la familia era testigo del paso de decenas de coronas de flores que llegaban incesa

In [29]:
with Pool(4) as p:
    res = p.map(func_process_story, dat.iterrows())

In [30]:
output = []
for story, questions, answers in res:
    if story is not None:
        for q, a in zip(questions, answers):
            output.append({'story': story, 'question': q, 'answer': a})

In [31]:
out_df = pd.DataFrame(output)

In [32]:
out_df.head()

,answer,question,story
0,@entity2,La información proviene de los documentos del...,La @entity0 ( @entity0) trabaja en la construc...
1,@entity10,@placeholder y su marido fueron asesinados en...,Esta semana @entity12 ha recibido una noticia ...
2,@entity1,Han detenido a 7 personas intregrantes de la ...,Esta semana @entity12 ha recibido una noticia ...
3,@entity12,@placeholder tiene una gran tasa de criminali...,Esta semana @entity12 ha recibido una noticia ...
4,@entity3,"Su líder moral, el dr. @placeholder, dice en ...",La situación en el Estado @entity14 de @entity...


In [33]:
out_df.shape

(78928, 3)

In [34]:
out_df.to_pickle("../data/es_qa_gcloud_final.pkl")